# Study of Full Outer Joins in *pandas*
*Eric Nordstrom* | *eanord4@gmail.com*  
*December 15th, 2019*

## I. Objective

I recently found that, in addition to the standard *pd.merge* and *pd.DataFrame.merge* functions, *pandas* provides at least one other way to perform a full outer join, which is to create a *pd.DataFrame* instance and feed it *pd.Series* instances as data (see **Introduction**).

This experiment aims to determine the relative speed and scaling between these three ways of joining with *pandas*. The experiment also aims to find how null values affect the speed of joining.

## II. Introduction

The *pandas* Python module has at least the following three ways of performing joins between datasets.

1. ***pandas.merge*** is a function taking inputs of two *pandas.DataFrame* instances (datasets) in addition to parameters specifying the type of join, the column(s) on which to join, etc.

2. ***pandas.DataFrame.merge*** is a method of the *pandas.DataFrame* class which considers the instance whose *merge* method was called as the **left** dataset and an input data frame as the **right** dataset. Additional parameters are used similarly to *pandas.merge*. This method likely works in the same way as *pandas.merge*, but this was not confirmed before the experiment.

3. Creating a new data frame by calling the ***pandas.DataFrame*** class (thereby calling its *\__new\__* method) also performs a full outer join if *pandas.Series* instances are provided as columns. The join in this case is assumed to be on the indices of the series provided as illustrated in the following example:

```
In:  pd.DataFrame({
         'A': pd.Series([1,2,3], index=[1,2,3]),
         'B': pd.Series(['a', 'b', 'c'], index=[2, 5, 9])
     })

Out:      A    B
     1  1.0  NaN
     2  2.0    a
     3  3.0  NaN
     5  NaN    b
     9  NaN    c
```

It is unknown at this time what algorithm each of the above methods uses. However, a hypothesized algorithm is provided in **§ III.A**. Assuming this algorithm for all three methods, the time required for joining should scale as

$
O[a(L_\text{left}-n_\text{left}+L_\text{right}-n_\text{right}) + b(n_\text{left}+n_\text{right})]\\
=O[a(L_\text{left}+L_\text{right}) + (b-a)(N_\text{left}+N_\text{right})]
$

where $L_\text{[side]}$ is the length of the original dataset and $N_\text{[side]}$ is the number of null values resulting on the given side. Constants $a$ and $b$ are unknown and represent the fact that shared indices and unshared indices might require different amounts of time to handle. Using $L$ as the single original dataset length and $n$ as the fraction of rows which are unshared, the expression becomes

$ O[a(2L) + (a-b)(2nL)] $

or

(1) $\qquad O[(\alpha + \beta n)L]$,

where $\alpha$ and $\beta$ are unknown constants and the sign of $\beta$ informs whether the join is sped up or slowed down by null values. From here, a linear model of the timing can be created as follows:

(2) $\qquad \hat{t}(L) = t_0 + \hat{m}L$,

where $\hat{m}$ itself is modeled linearly as follows:

(3) $\qquad \hat{m}(n) = \alpha + \beta n$

## III. Notes

### A. Notes about the "length" variable
It is not known which "length" is related to the time required to join datasets. This would depend on the exact algorithm being used to compare indices. However, a reasonable algorithm for a full outer join might be like the following:
1. Iterate over the left indices. For each index, check if it is present in the right dataset.
    * If it is, include it in the resulting data frame. **Note to ignore this row when iterating over the right dataset.**
    * If it is not, place a NaN value in the column representing the right dataset.
2. Iterate over the right indices. **Skip previously used rows.** 
    * The remaining rows cannot be present in the left dataset since it was already iterated over. Therefore, for each row, place a *NaN* value in the colum representing the left dataset.

### B. Notes about statistical methods

*At this time, it is not known which statistical methods are appropriate to calculate the above models in a way that reflects their influence on each other. This will be researched after data is gathered.*

## IV. Methods

This notebook is runnable and involves random data. Therefore, it will give slightly different results each time. The full procedure takes between 30 minutes and an hour on my laptop for the cases described. If one wishes to test it with different cases, one can feel free to edit the **Initialize experimental data** section or tweak anything else desired.

For greatest precision of results, it is best to stop other processes on one's computer to avoid interference with processing times. Alternatively, one can run the notebook alongside other processes to simulate a real-world situation.

My results will also be provided at https://github.com/eanord4/random.git in CSV format. The **Choose data source** code block below allows the user to select between reading from a CSV and recalculating the experimental data.

In the procedure below, functions are first defined to systematize the creation of datasets and the way joins were performed. For both the "file" and "recalculate" options, example datasets and joins are displayed for convenience after the function definitions.

Each join is performed on two similar datasets of the same length. Values are randomly selected *floats* between 0 and 1. Each dataset consists of a single column of values and a set of integer indices upon which to perform the join. Each join is a full outer join on the indices.

Next, experimental data are collected under each case described below. For each combination of $L$ and $n$, six sets of trials are performed--one in each possible order of the three join methods. The order of the six sets of trials is randomized before collection of data (but kept constant throughout collection). In each trial, the join is performed three times within the *timeit* function.

Once collected, all trials are analyzed using *pandas* and *matplotlib* plotting methods.

### A. Cases
* data frame length ($L$) = 10<sup>$p$</sup> where $p$ ranges from 1 to 7 in increments of 1
    * *It is not yet clear which "length" is related to the time required for the join. See notes below.*
* number of unshared rows (resulting in null values) ($n$) = 10% to 90% in increments of 10%
* join methods used: *pandas.merge*, *pandas.DataFrame.merge*, and *pandas.DataFrame.\__new\__*

### B. Dependencies
The following Python packages were used:
* *pandas* 0.25.3
* *matplotlib* 3.1.2
* *scikit-learn* 0.19.1
* *random*
* *timeit*
* *math*
* *itertools*

### C. Statistical methods
See **§ III.B**.

## V. Procedure

### A. Setup

#### 1. Choose data source

In [1]:
# Determine data source
data_source_choices = ['from file', 'recalculate']
data_source = data_source_choices[int(input('''\
Choose data source (type 0 or 1)...
    "from file" = 0
    "recalculate" = 1
Input: \
'''))]

# Establish file path

default_csv_path = 'pandas_join_comparison_results.csv'  # default CSV path
action = 'save' if data_source == 'recalculate' else 'retrieve'
user_input = input(f'''\
Enter CSV path to {action} data...
    or press <Enter> for default: "{default_csv_path}"
Input: \
''')
_csv_path = user_input if user_input.split() else default_csv_path

if data_source == 'recalculate':
    output_csv_path = _csv_path
else:
    input_csv_path = _csv_path

Choose data source (type 0 or 1)...
    "from file" = 0
    "recalculate" = 1
Input: 0
Enter CSV path to retrieve data...
    or press <Enter> for default: "pandas_join_comparison_results.csv"
Input: 


#### 2. Import dependencies

In [2]:
from timeit import timeit
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
import pandas as pd
import random as rm

if data_source == 'recalculate':
    import math
    import itertools as it

#### 3. Define useful functions to produce datasets and joins

In [3]:
def pd_merge(left_series, right_series):
    '''merge the datasets using `pd.merge`'''

    return pd.merge(left_series, right_series, how='outer', left_index=True, right_index=True)

def df_merge(left_df, right_series):
    '''merge the datasets using `pd.DataFrame.merge`; requires left dataset to be a data frame'''

    return left_df.merge(right_series, how='outer', left_index=True, right_index=True)

def new_df_join(left_series, right_series):
    '''merge the datasets by calling `pd.DataFrame`'''

    return pd.DataFrame({'LEFT': left_series, 'RIGHT': right_series})

def datasets(L, n):
    '''generate two series of the specified length and unshared fraction of rows'''

    N_unshared = int(n * L)  # number of unshared rows in each dataset. The total number of null values after joining will be double.
    length_range = range(L)
    all_indices = range(L + N_unshared)

    # get values
    left_data = [rm.random() for i in length_range]
    right_data = [rm.random() for i in length_range]

    # get left indices
    left_indices = sorted(rm.sample(all_indices, L))
    remaining = set(all_indices).difference(left_indices)

    # get right indices
    right_indices = rm.sample(left_indices, L - N_unshared)  # shared indices
    right_indices += rm.sample(remaining, N_unshared)  # unshared indices
    right_indices = sorted(right_indices)  # order of shared and unshared will be random

    # construct datasets as series
    left_series = pd.Series(left_data, index=left_indices, name='LEFT')
    right_series = pd.Series(right_data, index=right_indices, name='RIGHT')

    return left_series, right_series

#### 4. Test the functions and display example datasets and join results

##### i. Get example datasets

In [4]:
L = 10
n = .5
left_example, right_example = datasets(L, n)

##### ii. Display left example dataset

In [5]:
left_example

1     0.813005
4     0.638027
5     0.837428
7     0.203164
9     0.773284
10    0.782185
11    0.828333
12    0.129063
13    0.034586
14    0.598717
Name: LEFT, dtype: float64

##### iii. Display right example dataset

In [6]:
right_example

0     0.568928
2     0.260085
3     0.488607
5     0.609170
6     0.934339
8     0.863590
9     0.401285
10    0.246014
11    0.109273
13    0.371681
Name: RIGHT, dtype: float64

##### iv. Display example join using pandas.merge

In [7]:
pd_merge_result = pd_merge(left_example, right_example)
pd_merge_result

,LEFT,RIGHT
0,NaN,0.568928
1,0.813005,NaN
2,NaN,0.260085
3,NaN,0.488607
4,0.638027,NaN
5,0.837428,0.609170
6,NaN,0.934339
7,0.203164,NaN
8,NaN,0.863590
9,0.773284,0.401285


##### v. Show that all ways of joining yield the same result

In [8]:
print('`pd.merge` result same as `pd.DataFrame.merge`?')
print('---------------------------------------------------')
print(all( pd_merge_result == df_merge(left_example.to_frame(), right_example) ))
print()

print('`pd.merge` result same as `pd.DataFrame.__new__`?')
print('---------------------------------------------------')
print(all( pd_merge_result == new_df_join(left_example, right_example) ))

`pd.merge` result same as `pd.DataFrame.merge`?
---------------------------------------------------
True

`pd.merge` result same as `pd.DataFrame.__new__`?
---------------------------------------------------
True


### B. Experiment

#### 1. If recalculating: initialize experimental data

In [9]:
if data_source == 'recalculate':
    
    # Cases to be tried
    L_cases = [10, 10 ** 2, 10 ** 3, 10 ** 4, 10 ** 5, 10 ** 6, 10 ** 7]
    n_cases = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
    join_names = ['pd.merge', 'pd.DataFrame.merge', 'pd.DataFrame.__new__']
    join_funcs = [pd_merge, df_merge, new_df_join]

    # Additional definitions
    n_as_pcts = [f'{100 * n:.1f}%' for n in n_cases]
    join_cases = list(zip(join_names, join_funcs))
    join_permutations = list(it.permutations(join_cases))
    num_methods = len(join_cases)
    num_trials = len(join_permutations)
    num_to_time = 3  # repetitions within `timeit`

    # Initialize
    times = pd.DataFrame(

        columns=join_names,

        index=pd.MultiIndex.from_product(
            [L_cases, n_as_pcts, range(num_trials)],
            names=['L', 'n', 'Trial']
        )

    )

    # Show structure of experimental data
    times.head(12)

#### 2. Obtain data

In [10]:
if data_source == 'recalculate':

    # Randomize order of sets of trials

    randomized_trials = rm.sample(join_permutations, num_trials)


    # Collect data

    for L in L_cases:

        print(f'L = {L:,}...')

        for n, n_as_pct in zip(n_cases, n_as_pcts):

            print(f'\tn = {n_as_pct}...')

            for trial_num, permutation in enumerate(randomized_trials):
                print(f'\t\tTrial #{trial_num}...')

                for join_name, join_func in permutation:

                    print(f'\t\t\t{join_name}...')

                    times.loc[(L, n_as_pct, trial_num), join_name] = timeit(

                        stmt='join_func(left_data, right_data)',

                        setup='left_data, right_data = datasets(L, n)'
                        + '\nleft_data = left_data.to_frame()'
                        * (join_name == 'pd.DataFrame.merge'),

                        number=num_to_time,

                        globals={
                            'join_func': join_func,
                            'datasets': datasets,
                            'L': L,
                            'n': n,
                        }

                    ) / num_to_time

            print()

        print()
        
        
    # Save data to new file

    times.to_csv(output_csv_path)

else:
    times = pd.read_csv(input_csv_path)

### C. Analysis

#### 1. Preview data

In [11]:
times

,L,n,Trial,pd.merge,pd.DataFrame.merge,pd.DataFrame.__new__
0,10,10.0%,0,0.001852,0.001651,0.001046
1,10,10.0%,1,0.001687,0.001327,0.001229
2,10,10.0%,2,0.001624,0.001237,0.001025
3,10,10.0%,3,0.001575,0.001379,0.000892
4,10,10.0%,4,0.001961,0.001288,0.000911
...,...,...,...,...,...,...
373,10000000,90.0%,1,2.092372,2.046584,3.797931
374,10000000,90.0%,2,2.062062,1.998068,3.781070
375,10000000,90.0%,3,2.131902,2.014894,3.755780
376,10000000,90.0%,4,2.084182,1.981168,3.675436


#### 2. Plot time vs. length

In [12]:
# times.xs(.5, level=1)\
# .apply(lambda s: s.apply(math.log))\
# .reset_index(level=0)\
# .plot.line(x='L')